# MAR 653 Marketing Analytics
## 


Ryan Timbrook (RTIMBROO)  
DATE:
Topic: 

## 1. Objective
_____________________________________________________________________________________________



______________________________________________________________________________________________
### Coding Environment Setup
Import packages

In [2]:
# import packages for analysis and modeling
import pandas as pd #data frame operations
import numpy as np #arrays and math functions

## -- VISUALIZATION Packages
import matplotlib.pyplot as plt #2D plotting
%matplotlib inline

plt.style.use('fivethirtyeight')
import seaborn as sns #
## -- 
from collections import OrderedDict 
import os
import io
import re
from itertools import product
from datetime import date
from datetime import time
from datetime import datetime
import warnings
from timeit import default_timer               # performance processing time
import logging                                 # logging framework
# get loggers
warnings.filterwarnings('ignore')

from collections import Counter
from scipy.stats.stats import pearsonr
from string import ascii_letters

# Some helper functions to make our plots cleaner with Plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
init_notebook_mode(connected=True)

pd.set_option('display.max_columns', 500)


In [3]:
# custome python packages
import rtimbroo_utils as rt             # custome python helper functions
import auq_42_utils as au

All the files are downloaded


In [4]:
# set global properties
notebook_file_name = 'airbnb_seattle_listing_dimensions'
report_file_name = 'airbnb_seattle_listing_dimensions'
app_name = 'airbnb_seattle_listing_dimensions'
log_level = 10 # 10-DEBUG, 20-INFO, 30-WARNING, 40-ERROR, 50-CRITICAL

# setup working directory structure
# set global properties
dataDir = './data'
outputDir = './output'
configDir = './config'
logOutDir = './logs'
imageDir = './images'
modelDir = './models'
corpusDir = './corpus'

In [5]:
# create base output directories if they don't exist
if not os.path.exists(outputDir): os.mkdir(outputDir)
if not os.path.exists(logOutDir): os.mkdir(logOutDir)
if not os.path.exists(imageDir): os.mkdir(imageDir)
if not os.path.exists(modelDir): os.mkdir(modelDir)
if not os.path.exists(corpusDir): os.mkdir(corpusDir)
    
cleanPath = f'{dataDir}/clean'
if not os.path.exists(cleanPath): os.makedirs(cleanPath)

In [6]:
# get current date
now = datetime.utcnow().isoformat()
now_date = re.findall('^[0-9]{4}-[0-9]{2}-[0-9]{2}',now)
now_date

['2020-03-18']

In [7]:
#a logger for troubleshooting / data exploration
logger = rt.getFileLogger(logOutDir+'/',app_name+'_'+now_date[0],level=log_level)

## EDA Functions

In [8]:
#Function for showing columns with missing values
def show_missing_values(df):
    missing_vals = df.isnull().sum().sort_values(ascending = False)
    
    return missing_vals.iloc[missing_vals.nonzero()[0]]

In [9]:
def rangeScore(x):
    '''
    Set the bins for the score-range.
    '''
    value = ''
    if (x>= 0 and x < 10):
        value = '0-10'
    elif (x>= 10 and x < 20):
        value = '10-20'
    elif (x>= 20 and x < 30):
        value = '20-30'
    elif (x>= 30.0 and x < 40.0):
        value = '30-40'
    elif (x>= 40 and x < 50):
        value = '40-50'
    elif (x>= 50 and x < 60):
        value = '50-60'
    elif (x>= 60 and x < 70):
        value = '60-70'        
    elif (x>= 70 and x < 80):
        value = '70-80'
    elif (x>= 80 and x < 90):
        value = '80-90'
    elif (x>= 90 and x < 100):
        value = '90-100'
    elif x>= 100:
        value = '100+'
        
    return value

def convert_to_binary(x):
    if x == 't':
        return 1
    elif x == 'f':
        return 0
    else:
        return 0
    

In [10]:
#Create dummy columns by one-hot encoding
def create_dummies(df, columns = ['room_type', 'property_type', 'bed_type', 'cancellation_policy']):
    for column in columns:
        dummies = pd.get_dummies(df[column], prefix = column)
        df = pd.concat([df,dummies], axis = 1)
    return df

## 2. OBTAIN the data   
________________________________________________________________________________________________
Import external datasets for evaluation

##### Airbnb Seattle
* Customer Reviews: reviews.csv
* Host Listings: listings.csv
* Calendar Listings: calendar.csv

In [11]:
# datasets file names to load
host_listings = pd.read_csv(f'{dataDir}/seattle/listings.csv', error_bad_lines=False, encoding="ISO-8859-1")
#calendar_listings = pd.read_csv(f'{dataDir}/seattle/calendar.csv', error_bad_lines=False, encoding="ISO-8859-1")

In [12]:
host_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
id                                  3818 non-null int64
listing_url                         3818 non-null object
scrape_id                           3818 non-null int64
last_scraped                        3818 non-null object
name                                3818 non-null object
summary                             3641 non-null object
space                               3249 non-null object
description                         3818 non-null object
experiences_offered                 3818 non-null object
neighborhood_overview               2786 non-null object
notes                               2212 non-null object
transit                             2884 non-null object
thumbnail_url                       3498 non-null object
medium_url                          3498 non-null object
picture_url                         3818 non-null object
xl_picture_url                      3498

In [13]:
temp_dummies = create_dummies(host_listings,['host_is_superhost',
                                             'host_identity_verified',
                                             'host_has_profile_pic',
                                             'neighbourhood_group_cleansed',
                                             'room_type', 
                                             'property_type', 
                                             'bed_type', 
                                             'cancellation_policy'])
temp_dummies.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,host_is_superhost_f,host_is_superhost_t,host_identity_verified_f,host_identity_verified_t,host_has_profile_pic_f,host_has_profile_pic_t,neighbourhood_group_cleansed_Ballard,neighbourhood_group_cleansed_Beacon Hill,neighbourhood_group_cleansed_Capitol Hill,neighbourhood_group_cleansed_Cascade,neighbourhood_group_cleansed_Central Area,neighbourhood_group_cleansed_Delridge,neighbourhood_group_cleansed_Downtown,neighbourhood_group_cleansed_Interbay,neighbourhood_group_cleansed_Lake City,neighbourhood_group_cleansed_Magnolia,neighbourhood_group_cleansed_Northgate,neighbourhood_group_cleansed_Other neighborhoods,neighbourhood_group_cleansed_Queen Anne,neighbourhood_group_cleansed_Rainier Valley,neighbourhood_group_cleansed_Seward Park,neighbourhood_group_cleansed_University District,neighbourhood_group_cleansed_West Seattle,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,property_type_Bungalow,property_type_Cabin,property_type_Camper/RV,property_type_Chalet,property_type_Condominium,property_type_Dorm,property_type_House,property_type_Loft,property_type_Other,property_type_Tent,property_type_Townhouse,property_type_Treehouse,property_type_Yurt,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,NaN,956883,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96%,100%,f,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.636289,-122.371025,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$85.00,NaN,NaN,NaN,NaN,2,$5.00,1,365,4 weeks ago,t,14,41,71,346,2016-01-04,207,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,953595,

In [14]:
host_listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [15]:
cols = list(host_listings.columns)
cols.remove('id')

temp_dummies = temp_dummies.drop(cols, axis=1)
temp_dummies = temp_dummies.rename(columns={"id":"listing_id"})

temp_dummies.to_csv(f'{cleanPath}/temp_dummies.csv', index=False)

logger.info(f'{temp_dummies.shape}')
logger.info(f'{temp_dummies.columns}')

(3818, 51)
Index(['listing_id', 'host_is_superhost_f', 'host_is_superhost_t',
       'host_identity_verified_f', 'host_identity_verified_t',
       'host_has_profile_pic_f', 'host_has_profile_pic_t',
       'neighbourhood_group_cleansed_Ballard',
       'neighbourhood_group_cleansed_Beacon Hill',
       'neighbourhood_group_cleansed_Capitol Hill',
       'neighbourhood_group_cleansed_Cascade',
       'neighbourhood_group_cleansed_Central Area',
       'neighbourhood_group_cleansed_Delridge',
       'neighbourhood_group_cleansed_Downtown',
       'neighbourhood_group_cleansed_Interbay',
       'neighbourhood_group_cleansed_Lake City',
       'neighbourhood_group_cleansed_Magnolia',
       'neighbourhood_group_cleansed_Northgate',
       'neighbourhood_group_cleansed_Other neighborhoods',
       'neighbourhood_group_cleansed_Queen Anne',
       'neighbourhood_group_cleansed_Rainier Valley',
       'neighbourhood_group_cleansed_Seward Park',
       'neighbourhood_group_cleansed_University

In [16]:
host_listings.shape

(3818, 92)

In [17]:
host_listings.cancellation_policy.unique()

array(['moderate', 'strict', 'flexible'], dtype=object)

In [18]:
pt = pd.DataFrame(data=list(host_listings.property_type.unique()))
pt.to_csv(f'{cleanPath}/property_types.csv', index=False)

## Split Listings data into 

### ListingDescriptionText
* id
* name
* summary
* space
* description
* experiences_offered
* neighborhood_overview
* notes
* transit

In [19]:
host_listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

# Listing Description Text

In [20]:
listing_description_text = host_listings[['id','name','summary','space','description','experiences_offered','neighborhood_overview','notes','transit','host_about']]
show_missing_values(listing_description_text)

notes                    1606
neighborhood_overview    1032
transit                   934
host_about                859
space                     569
summary                   177
dtype: int64

In [21]:
# Create Listing Description Text Object

listing_description_text['name'] = listing_description_text['name'].fillna('UNKNOWN')
listing_description_text['summary'] = listing_description_text['summary'].fillna('UNKNOWN')
listing_description_text['space'] = listing_description_text['space'].fillna('UNKNOWN')
listing_description_text['description'] = listing_description_text['description'].fillna('UNKNOWN')
listing_description_text['experiences_offered'] = listing_description_text['experiences_offered'].fillna('UNKNOWN')
listing_description_text['neighborhood_overview'] = listing_description_text['neighborhood_overview'].fillna('UNKNOWN')
listing_description_text['notes'] = listing_description_text['notes'].fillna('UNKNOWN')
listing_description_text['transit'] = listing_description_text['transit'].fillna('UNKNOWN')
listing_description_text['host_about'] = listing_description_text['host_about'].fillna('UNKNOWN')

listing_description_text = listing_description_text.rename(columns={"id":"listing_id","name":"listing_name","notes":"listing_notes"})
#listing_description_text.head()


# Listing Location

In [22]:
# Create Listing Location Object
listing_location = host_listings[['id','host_id','street','neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact']]

show_missing_values(listing_location)

zipcode    7
dtype: int64

In [23]:
# clean listings
listing_location['street'] = listing_location['street'].fillna('UNKNOWN')
listing_location['neighbourhood_cleansed'] = listing_location['neighbourhood_cleansed'].fillna('UNKNOWN')
listing_location['neighbourhood_group_cleansed'] = listing_location['neighbourhood_group_cleansed'].fillna('UNKNOWN')
listing_location['city'] = listing_location['city'].fillna('UNKNOWN')
listing_location['state'] = listing_location['state'].fillna('UNKNOWN')
listing_location['zipcode'] = listing_location['zipcode'].fillna('UNKNOWN')
listing_location['market'] = listing_location['market'].fillna('UNKNOWN')
listing_location['smart_location'] = listing_location['smart_location'].fillna('UNKNOWN')
listing_location['country_code'] = listing_location['country_code'].fillna('UNKNOWN')
listing_location['country'] = listing_location['country'].fillna('UNKNOWN')
listing_location['latitude'] = listing_location['latitude'].fillna('UNKNOWN')
listing_location['longitude'] = listing_location['longitude'].fillna('UNKNOWN')
listing_location['is_location_exact'] = listing_location['is_location_exact'].fillna('f')


listing_location = listing_location.rename(columns={"id":"listing_id",
                                                    "neighbourhood_cleansed":"neighbourhood",
                                                    "neighbourhood_group_cleansed":"neighbourhood_group"}
                                                    )

#listing_location.head()


In [24]:
# get unique addresses
def clean_street(s):
    # UPPERCASE
    s = s.upper()
    # split on ,
    t = s.split(',')
    # get ride of city,state,zip, country, from this field
    street = t[0]
    if len(street) < 2: 
        street = 'UNKNOWN'
    return street
    

In [25]:
listing_location['street_cleaned'] = [clean_street(s) for s in listing_location['street']]

#listing_location['street_cleaned'].unique
host_property_locations = listing_location.groupby(['street_cleaned','host_id']).filter(lambda g: len(g)>1).drop_duplicates(subset=['host_id','street_cleaned','zipcode'],keep='first')

host_property_locations.shape

(420, 16)

In [26]:
host_property_locations = host_property_locations.drop(columns=['listing_id'])

In [27]:

show_missing_values(host_property_locations)
#host_property_locations.head()

Series([], dtype: int64)

In [28]:
# save as data object
host_property_locations.to_csv(f'{cleanPath}/host_property_locations.csv', index=False)


# Listing Property Features

In [29]:
# Create Listing Property Features
listing_property_features = host_listings[['id','property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet']]

show_missing_values(listing_property_features)

square_feet      3721
bathrooms          16
bedrooms            6
beds                1
property_type       1
dtype: int64

In [30]:
# clean listing_property_features

listing_property_features['square_feet'] = listing_property_features['square_feet'].fillna('0')
listing_property_features['amenities'] = listing_property_features['amenities'].fillna('UNKNOWN')
listing_property_features['bed_type'] = listing_property_features['bed_type'].fillna('UNKNOWN')
listing_property_features['beds'] = listing_property_features['beds'].fillna(0)
listing_property_features['bedrooms'] = listing_property_features['bedrooms'].fillna(0)
listing_property_features['bathrooms'] = listing_property_features['bathrooms'].fillna(0)
listing_property_features['accommodates'] = listing_property_features['accommodates'].fillna(0)
listing_property_features['room_type'] = listing_property_features['room_type'].fillna('UNKNOWN')
listing_property_features['property_type'] = listing_property_features['property_type'].fillna('UNKNOWN')

#Calculate the bedroom and bathroom share per person. Higher the share, more the comfort.
listing_property_features['bedroom_share'] = round(listing_property_features['bedrooms']/listing_property_features['accommodates'],2)
listing_property_features['bathroom_share'] = round(listing_property_features['bathrooms']/listing_property_features['accommodates'],2)

listing_property_features = listing_property_features.rename(columns={"id":"listing_id"})
listing_property_features.head()

,listing_id,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,bedroom_share,bathroom_share
0,241032,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",0,0.25,0.25
1,953595,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0,0.25,0.25
2,3308979,House,Entire home/apt,11,4.5,5.0,7.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",0,0.45,0.41
3,7421966,Apartment,Entire home/apt,3,1.0,0.0,2.0,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",0,0.00,0.33
4,278830,House,Entire home/apt,6,2.0,3.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",0,0.50,0.33


In [31]:
listing_property_features['property_type'].unique()

array(['Apartment', 'House', 'Cabin', 'Condominium', 'Camper/RV',
       'Bungalow', 'Townhouse', 'Loft', 'Boat', 'Bed & Breakfast',
       'Other', 'Dorm', 'Treehouse', 'Yurt', 'Chalet', 'Tent', 'UNKNOWN'],
      dtype=object)

In [32]:
listing_property_features['room_type'].unique()
#rt = pd.DataFrame(data=list(listing_property_features['room_type'].unique()))
#rt.to_csv(f'{cleanPath}/room_types.csv', index=False)

array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object)

In [33]:
# create dummy variables for room types
private_rooms = []
shared_rooms = []
entire_home_apts = []

for r in listing_property_features['room_type']:
    if r == 'Entire home/apt':
        private_rooms.append(1)
    else:
        private_rooms.append(0)
        
    if r == 'Private room':
        shared_rooms.append(1)
    else:
        shared_rooms.append(0)
        
    if r == 'Shared room':
        entire_home_apts.append(1)
    else:
        entire_home_apts.append(0)

listing_property_features['private_room'] = private_rooms
listing_property_features['shared_room'] = shared_rooms
listing_property_features['entire_home_apt'] = entire_home_apts

listing_property_features.head()

,listing_id,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,bedroom_share,bathroom_share,private_room,shared_room,entire_home_apt
0,241032,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",0,0.25,0.25,1,0,0
1,953595,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0,0.25,0.25,1,0,0
2,3308979,House,Entire home/apt,11,4.5,5.0,7.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",0,0.45,0.41,1,0,0
3,7421966,Apartment,Entire home/apt,3,1.0,0.0,2.0,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",0,0.00,0.33,1,0,0
4,278830,House,Entire home/apt,6,2.0,3.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",0,0.50,0.33,1,0,0


In [34]:
listing_property_features['bed_type'].unique()

array(['Real Bed', 'Futon', 'Pull-out Sofa', 'Airbed', 'Couch'],
      dtype=object)

In [35]:
# create dummy variables for bed types

real_beds = []
futon_beds = []
pull_out_sofa_beds = []
air_beds = []
couch_beds = []

for b in listing_property_features['bed_type']:
    if b == 'Real Bed':
        real_beds.append(1)
    else:
        real_beds.append(0)
        
    if b == 'Futon':
        futon_beds.append(1)
    else:
        futon_beds.append(0)
        
    if b == 'Pull-out Sofa':
        pull_out_sofa_beds.append(1)
    else:
        pull_out_sofa_beds.append(0)
        
    if b == 'Airbed':
        air_beds.append(1)
    else:
        air_beds.append(0)
        
    if b == 'Couch':
        couch_beds.append(1)
    else:
        couch_beds.append(0)
        
listing_property_features['real_bed'] = real_beds
listing_property_features['futon_bed'] = futon_beds
listing_property_features['pull_out_sofa_bed'] = pull_out_sofa_beds
listing_property_features['air_bed'] = air_beds
listing_property_features['couch_bed'] = couch_beds

listing_property_features.head()

,listing_id,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,bedroom_share,bathroom_share,private_room,shared_room,entire_home_apt,real_bed,futon_bed,pull_out_sofa_bed,air_bed,couch_bed
0,241032,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",0,0.25,0.25,1,0,0,1,0,0,0,0
1,953595,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0,0.25,0.25,1,0,0,1,0,0,0,0
2,3308979,House,Entire home/apt,11,4.5,5.0,7.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",0,0.45,0.41,1,0,0,1,0,0,0,0
3,7421966,Apartment,Entire home/apt,3,1.0,0.0,2.0,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",0,0.00,0.33,1,0,0,1,0,0,0,0
4,278830,House,Entire home/apt,6,2.0,3.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",0,0.50,0.33,1,0,0,1,0,0,0,0


In [36]:
listing_property_features.to_csv(f'{cleanPath}/listing_property_features.csv', index=False)

In [37]:
features_numeric = ['listing_id','accommodates','bathrooms','bedrooms','beds','bedroom_share','bathroom_share']
occupancy_features = listing_property_features[features_numeric]
occupancy_features.to_csv(f'{cleanPath}/occupancy_features.csv', index=False)

In [38]:
sub_feats = ['listing_id','private_room','shared_room','entire_home_apt','real_bed','futon_bed','pull_out_sofa_bed','air_bed','couch_bed']
listing_property_sub_features = listing_property_features[sub_feats]
#listing_property_sub_features.head()
feats_table = listing_property_sub_features.melt(id_vars=['listing_id'], var_name='property_feature')
feats_filtered = feats_table[feats_table['value']==1]
feats_filtered.to_csv(f'{cleanPath}/listing_property_sub_features.csv', index=False)

In [39]:
#TODO-- break amenities into it's own object into it's 
listing_property_features['amenities'][:5]

0    {TV,"Cable TV",Internet,"Wireless Internet","A...
1    {TV,Internet,"Wireless Internet",Kitchen,"Free...
2    {TV,"Cable TV",Internet,"Wireless Internet","A...
3    {Internet,"Wireless Internet",Kitchen,"Indoor ...
4    {TV,"Cable TV",Internet,"Wireless Internet",Ki...
Name: amenities, dtype: object

# Listing Property Types

In [40]:
# property types
listing_ids = listing_property_features['listing_id']
apartments = []
houses = []
cabins = []
condominiums = []
campers_rvs = []
bungalows = []
townhouses = []
lofts = []
boats = []
bed_breakfasts = []
dorms = []
treehouses = []
yurts = []
chalets = []
tents = []
others = []
unknowns = []

for p in listing_property_features['property_type']:
    if p == 'Apartment':
        apartments.append(1)
    else:
        apartments.append(0)
    
    if p == 'House':
        houses.append(1)
    else:
        houses.append(0)
        
    if p == 'Cabin':
        cabins.append(1)
    else:
        cabins.append(0)
        
    if p == 'Condominium':
        condominiums.append(1)
    else:
        condominiums.append(0)
        
    if p == 'Camper/RV':
        campers_rvs.append(1)
    else:
        campers_rvs.append(0)
        
    if p == 'Bungalow':
        bungalows.append(1)
    else:
        bungalows.append(0)
        
    if p == 'Townhouse':
        townhouses.append(1)
    else:
        townhouses.append(0)
        
    if p == 'Loft':
        lofts.append(1)
    else:
        lofts.append(0)
        
    if p == 'Boat':
        boats.append(1)
    else:
        boats.append(0)
        
    if p == 'Bed & Breakfast':
        bed_breakfasts.append(1)
    else:
        bed_breakfasts.append(0)
        
    if p == 'Other':
        others.append(1)
    else:
        others.append(0)
        
    if p == 'Dorm':
        dorms.append(1)
    else:
        dorms.append(0)
        
    if p == 'Treehouse':
        treehouses.append(1)
    else:
        treehouses.append(0)
        
    if p == 'Yurt':
        yurts.append(1)
    else:
        yurts.append(0)
        
    if p == 'Chalet':
        chalets.append(1)
    else:
        chalets.append(0)
        
    if p == 'Tent':
        tents.append(1)
    else:
        tents.append(0)
        
    if p == 'UNKNOWN':
        unknowns.append(1)
    else:
        unknowns.append(0)
        
listing_property_types = pd.DataFrame()
listing_property_types['listing_id'] = listing_ids
listing_property_types['apartment'] = apartments
listing_property_types['house'] = houses
listing_property_types['cabin'] = cabins
listing_property_types['condominium'] = condominiums
listing_property_types['campers_rv'] = campers_rvs
listing_property_types['bungalow'] = bungalows
listing_property_types['townhouse'] = townhouses
listing_property_types['loft'] = lofts
listing_property_types['boat'] = boats
listing_property_types['bed_breakfasts'] = bed_breakfasts
listing_property_types['dorm'] = dorms
listing_property_types['treehouse'] = treehouses
listing_property_types['yurt'] = yurts
listing_property_types['chalet'] = chalets
listing_property_types['tent'] = tents
listing_property_types['other'] = others
listing_property_types['unknown'] = unknowns


listing_property_types.head()

,listing_id,apartment,house,cabin,condominium,campers_rv,bungalow,townhouse,loft,boat,bed_breakfasts,dorm,treehouse,yurt,chalet,tent,other,unknown
0,241032,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,953595,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3308979,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,7421966,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,278830,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
listing_property_types.to_csv(f'{cleanPath}/listing_property_types.csv', index=False)

In [42]:
prop_type_table = listing_property_types.melt(id_vars=['listing_id'], var_name='property_type')
#prop_type_table.head()
prop_type_filtered = prop_type_table[prop_type_table['value']==1]
#prop_type_filtered.head()
prop_type_filtered.to_csv(f'{cleanPath}/listing_property_types.csv', index=False)

# Listing Amenities

In [43]:
def clean_amenities(a):
  
    a = a.replace('{','')
    a = a.replace('}','')
    a = a.replace('\"','')
    t = a.split(',')
  
    return t

def add_amenities(dic, a):
    if a in dic:
        dic[a] = 1

def format_cols(c):
    c = c.lower()
    c = c.replace(' ','_')
    c = c.replace('\\','_')
    c = c.replace('/','_')
    c = c.replace('(','')
    c = c.replace(')','')
    c = c.replace('-','')
    
    return c

In [44]:
# amenities
#listing_property_features['amenities']

amenities = []

for a in listing_property_features['amenities']:
    a = a.replace('{','')
    a = a.replace('}','')
    a = a.replace('\"','')
    t = a.split(',')
    #logger.info(t)
    
    for tok in t:
        if tok not in amenities and len(tok) > 1:
            amenities.append(tok)  
#amenities

In [45]:
logger.info(listing_property_features['amenities'][:1])

0    {TV,"Cable TV",Internet,"Wireless Internet","A...
Name: amenities, dtype: object


In [46]:
am = set()
for a in listing_property_features['amenities']:
    am.add(a)
am

{'{"Wireless Internet",Heating,Washer,Dryer,"Smoke Detector","Carbon Monoxide Detector",Essentials}',
 '{Internet,"Wireless Internet",Kitchen,"Free Parking on Premises",Cat(s),Heating,"Family/Kid Friendly",Washer,Dryer,Essentials,Shampoo}',
 '{TV,Internet,"Wireless Internet",Kitchen,"Free Parking on Premises","Hot Tub",Heating,"Family/Kid Friendly",Washer,Dryer,"Smoke Detector","Carbon Monoxide Detector","Fire Extinguisher",Essentials,Shampoo,"24-Hour Check-in",Hangers,"Hair Dryer",Iron,"Laptop Friendly Workspace"}',
 '{TV,"Cable TV",Internet,"Wireless Internet","Air Conditioning",Kitchen,"Free Parking on Premises",Heating,"Family/Kid Friendly",Washer,Dryer,"Smoke Detector","Carbon Monoxide Detector","Fire Extinguisher",Essentials,Shampoo,Hangers,Iron}',
 '{TV,"Cable TV",Internet,"Wireless Internet",Kitchen,"Free Parking on Premises","Pets Allowed",Heating,"Family/Kid Friendly",Washer,Dryer,"Smoke Detector","Carbon Monoxide Detector","First Aid Kit","Safety Card",Essentials,Shampoo}',


In [47]:
listing_ids = listing_property_features['listing_id']
listing_property_amenities = pd.DataFrame()

amenities_col = []

for e in listing_property_features['amenities']:
    #amenities_dic = OrderedDict()
    #for a in amenities:
        #amenities_dic[a] = 0
    amenities_dic = {a:0 for a in amenities}
    
    for a in clean_amenities(e):
        add_amenities(amenities_dic,a)
    
    amenities_col.append(amenities_dic)
    #break

#logger.debug(amenities_col)

In [48]:
amenities_col

[{'TV': 1,
  'Cable TV': 1,
  'Internet': 1,
  'Wireless Internet': 1,
  'Air Conditioning': 1,
  'Kitchen': 1,
  'Heating': 1,
  'Family/Kid Friendly': 1,
  'Washer': 1,
  'Dryer': 1,
  'Free Parking on Premises': 0,
  'Buzzer/Wireless Intercom': 0,
  'Smoke Detector': 0,
  'Carbon Monoxide Detector': 0,
  'First Aid Kit': 0,
  'Safety Card': 0,
  'Fire Extinguisher': 0,
  'Essentials': 0,
  'Pets Allowed': 0,
  'Pets live on this property': 0,
  'Dog(s)': 0,
  'Cat(s)': 0,
  'Hot Tub': 0,
  'Indoor Fireplace': 0,
  'Shampoo': 0,
  'Breakfast': 0,
  '24-Hour Check-in': 0,
  'Hangers': 0,
  'Hair Dryer': 0,
  'Iron': 0,
  'Laptop Friendly Workspace': 0,
  'Suitable for Events': 0,
  'Elevator in Building': 0,
  'Lock on Bedroom Door': 0,
  'Wheelchair Accessible': 0,
  'Gym': 0,
  'Pool': 0,
  'Smoking Allowed': 0,
  'Other pet(s)': 0,
  'Doorman': 0,
  'Washer / Dryer': 0},
 {'TV': 1,
  'Cable TV': 0,
  'Internet': 1,
  'Wireless Internet': 1,
  'Air Conditioning': 0,
  'Kitchen': 1,


In [49]:
cleaned_amenities_col = []
for col in amenities_col:
    
    for k in col.keys():
       
        new_key = format_cols(k)
        col[new_key] = col.pop(k)
     
    cleaned_amenities_col.append(col)
    
logger.debug(cleaned_amenities_col)

In [50]:
cleaned_amenities_df = pd.DataFrame(data=cleaned_amenities_col)
cleaned_amenities_df.rename(columns={'Internet':'internet',"Wireless Internet":'wireless_internet'})
cleaned_amenities_df['listing_id'] = listing_ids
cleaned_amenities_df.head()

,24hour_checkin,Internet,Wireless Internet,air_conditioning,breakfast,buzzer_wireless_intercom,cable_tv,carbon_monoxide_detector,cats,dogs,doorman,dryer,elevator_in_building,essentials,family_kid_friendly,fire_extinguisher,first_aid_kit,free_parking_on_premises,gym,hair_dryer,hangers,heating,hot_tub,indoor_fireplace,iron,kitchen,laptop_friendly_workspace,lock_on_bedroom_door,other_pets,pets_allowed,pets_live_on_this_property,pool,safety_card,shampoo,smoke_detector,smoking_allowed,suitable_for_events,tv,washer,washer___dryer,wheelchair_accessible,listing_id
0,0,1,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,241032
1,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,953595
2,0,1,1,1,0,0,1,1,1,1,0,1,0,1,1,0,0,1,0,0,0,1,1,1,0,1,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,3308979
3,0,1,1,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,7421966
4,0,1,1,0,0,0,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,278830


In [51]:
#cleaned_amenities_df.to_csv(f'{cleanPath}/listing_property_amenities.csv', index=False)

In [52]:
# datasets file names to load
cleaned_amenities_df = pd.read_csv(f'{cleanPath}/listing_property_amenities2.csv', error_bad_lines=False, encoding="ISO-8859-1")
cleaned_amenities_df = cleaned_amenities_df.rename(columns={'listing_ids':'listing_id'})
cleaned_amenities_df.head()

,listing_id,24hour_checkin,internet,wireless_internet,air_conditioning,breakfast,buzzer_wireless_intercom,cable_tv,carbon_monoxide_detector,cats,dogs,doorman,dryer,elevator_in_building,essentials,family_kid_friendly,fire_extinguisher,first_aid_kit,free_parking_on_premises,gym,hair_dryer,hangers,heating,hot_tub,indoor_fireplace,iron,kitchen,laptop_friendly_workspace,lock_on_bedroom_door,other_pets,pets_allowed,pets_live_on_this_property,pool,safety_card,shampoo,smoke_detector,smoking_allowed,suitable_for_events,tv,washer,washer_dryer,wheelchair_accessible
0,241032,0,1,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1,953595,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0
2,3308979,0,1,1,1,0,0,1,1,1,1,0,1,0,1,1,0,0,1,0,0,0,1,1,1,0,1,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0
3,7421966,0,1,1,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0
4,278830,0,1,1,0,0,0,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0


### Categorize amenities into meaningful groups

In [53]:
amenity_recode={
                'air_conditioning':'weather_control',
                'indoor_fireplace':'weather_control',
                'heating':'weather_control',
        
                'carbon_monoxide_detector':'safety_features',
                'fire_extinguisher':'safety_features',
                'first_aid_kit':'safety_features',
                'smoke_detector':'safety_features',
                
                'buzzer_wireless_intercom':'security_features',
                'doorman':'security_features',
                'safety_card':'security_features',
                'lock_on_bedroom_door':'security_features',
                
                'cats':'pet_friendly',
                'dogs':'pet_friendly',
                'pets_allowed':'pet_friendly',
                'pets_live_on_this_property':'pet_friendly',
                'other_pets':'pet_friendly',
                
                'elevator_in_building':'access_friendly',
                'wheelchair_accessible':'access_friendly',
                
                'essentials':'essentials',
                'hair_dryer':'essentials',
                'hangers':'essentials',
                'iron':'essentials',
                'shampoo':'essentials',             
                
                'cable_tv':'tv',
                'tv':'tv',
                
                'internet':'internet',
                'wireless_internet':'internet',
                'laptop_friendly_workspace':'internet',
                
                'dryer':'laundry_facility',
                'washer':'laundry_facility',
                'washer_dryer':'laundry_facility',
    
                #Leaving amenities as such which cannot be grouped
                #'Kitchen',
                #'Family/Kid Friendly', 
                #'Free Parking on Premises',
                #'Breakfast',
                #'24-Hour Check-in',
                #'Hot Tub',
                #'Pool',
                #'Gym',
                #'Smoking Allowed',
                #'Suitable for Events'
}

In [54]:
cleaned_amenities_df_melt = cleaned_amenities_df.melt(id_vars=['listing_id'], var_name='amenity')

#Recoding and putting in new column called amenity_modified
cleaned_amenities_df_melt = cleaned_amenities_df_melt.assign(
    amenity_modified = cleaned_amenities_df_melt.amenity.replace(amenity_recode)
)

cleaned_amenities_df_melt.tail()

,listing_id,amenity,value,amenity_modified
156533,8101950,wheelchair_accessible,1,access_friendly
156534,8902327,wheelchair_accessible,0,access_friendly
156535,10267360,wheelchair_accessible,0,access_friendly
156536,9604740,wheelchair_accessible,0,access_friendly
156537,10208623,wheelchair_accessible,0,access_friendly


In [55]:
#Pivot the melted dataframe before merging with original dataframe
cleaned_amenities_df_pivot = cleaned_amenities_df_melt.pivot_table(
    index='listing_id',
    columns='amenity_modified',
    values='value', 
    aggfunc='max'
)

cleaned_amenities_df_pivot.head()

amenity_modified,24hour_checkin,access_friendly,breakfast,essentials,family_kid_friendly,free_parking_on_premises,gym,hot_tub,internet,kitchen,laundry_facility,pet_friendly,pool,safety_features,security_features,smoking_allowed,suitable_for_events,tv,weather_control
listing_id,,,,,,,,,,,,,,,,,,,
3335,0,0,0,0,1,1,0,0,1,1,1,1,0,1,0,0,0,0,1
4291,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,1,1
5682,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,1
6606,0,0,0,1,0,1,0,1,1,1,0,1,0,0,0,0,0,1,1
7369,0,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,1,1


In [56]:
#Join the amenities dataframe back to the original listings dataframe
#cleaned_amenitie_joined=cleaned_amenities_df.join(cleaned_amenities_df_pivot,on="listing_ids",how="inner")
#cleaned_amenitie_joined.head()
cleaned_amenities_df_pivot.columns
cleaned_amenities_df_pivot = cleaned_amenities_df_pivot.reset_index()

cleaned_amenities_df_pivot.head()


amenity_modified,listing_id,24hour_checkin,access_friendly,breakfast,essentials,family_kid_friendly,free_parking_on_premises,gym,hot_tub,internet,kitchen,laundry_facility,pet_friendly,pool,safety_features,security_features,smoking_allowed,suitable_for_events,tv,weather_control
0,3335,0,0,0,0,1,1,0,0,1,1,1,1,0,1,0,0,0,0,1
1,4291,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,1,1
2,5682,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,1
3,6606,0,0,0,1,0,1,0,1,1,1,0,1,0,0,0,0,0,1,1
4,7369,0,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,1,1


In [57]:
amen_table = cleaned_amenities_df_pivot.melt(id_vars=['listing_id'], var_name='amenity')
amen_filtered = amen_table[amen_table['value']==1]
amen_filtered.to_csv(f'{cleanPath}/listing_property_amenities.csv', index=False)

In [58]:
cleaned_amenities_df_pivot.to_csv(f'{cleanPath}/listing_amenities_categorized.csv', index=False)


In [59]:
# join listing amenities with other dummy variables
temp_dummies2 = cleaned_amenities_df_pivot.merge(temp_dummies,how='inner',on='listing_id')

temp_dummies2.to_csv(f'{cleanPath}/temp_dummies2.csv', index=False)


In [60]:
logger.info(f'{temp_dummies2.shape}')

(3818, 70)


In [61]:
temp_dummies2.head()

,listing_id,24hour_checkin,access_friendly,breakfast,essentials,family_kid_friendly,free_parking_on_premises,gym,hot_tub,internet,kitchen,laundry_facility,pet_friendly,pool,safety_features,security_features,smoking_allowed,suitable_for_events,tv,weather_control,host_is_superhost_f,host_is_superhost_t,host_identity_verified_f,host_identity_verified_t,host_has_profile_pic_f,host_has_profile_pic_t,neighbourhood_group_cleansed_Ballard,neighbourhood_group_cleansed_Beacon Hill,neighbourhood_group_cleansed_Capitol Hill,neighbourhood_group_cleansed_Cascade,neighbourhood_group_cleansed_Central Area,neighbourhood_group_cleansed_Delridge,neighbourhood_group_cleansed_Downtown,neighbourhood_group_cleansed_Interbay,neighbourhood_group_cleansed_Lake City,neighbourhood_group_cleansed_Magnolia,neighbourhood_group_cleansed_Northgate,neighbourhood_group_cleansed_Other neighborhoods,neighbourhood_group_cleansed_Queen Anne,neighbourhood_group_cleansed_Rainier Valley,neighbourhood_group_cleansed_Seward Park,neighbourhood_group_cleansed_University District,neighbourhood_group_cleansed_West Seattle,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,property_type_Bungalow,property_type_Cabin,property_type_Camper/RV,property_type_Chalet,property_type_Condominium,property_type_Dorm,property_type_House,property_type_Loft,property_type_Other,property_type_Tent,property_type_Townhouse,property_type_Treehouse,property_type_Yurt,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict
0,3335,0,0,0,0,1,1,0,0,1,1,1,1,0,1,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,4291,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
2,5682,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,1,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,6606,0,0,0,1,0,1,0,1,1,1,0,1,0,0,0,0,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
4,7369,0,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,1,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


# Listing Price Fees

In [62]:
listing_calendar_available = pd.read_csv(f'{cleanPath}/listing_calendar_available.csv', error_bad_lines=False, encoding="ISO-8859-1")
listing_calendar_available.head()

,listing_id,calendar_available_days,avg_calendar_price
0,11411,365,60.0
1,10695,365,109.0
2,66540,365,65.0
3,10385,365,60.0
4,215954,365,87.0


In [63]:
listing_calendar_available.describe()

,listing_id,calendar_available_days,avg_calendar_price
count,3.723000e+03,3723.000000,3723.000000
mean,5.548051e+06,251.018533,135.764060
std,2.969790e+06,122.119707,97.385682
min,3.335000e+03,1.000000,20.000000
25%,3.242426e+06,139.000000,75.629042
50%,6.119821e+06,313.000000,108.416107
75%,8.036802e+06,360.000000,162.099432
max,1.034016e+07,365.000000,1301.818182


In [64]:
def convert_price(att):
    att = att.fillna(str(0))
    re_money = re.compile('^\$\d+')
    prices = []
    for e in att:
        e = str(e)
        e = e.replace('\'','')
        e = e.replace(',','')
        #print(e)
        if re.match(re_money,e):
            #print(e[1:])
            prices.append(e[1:])
        else:
            prices.append(e)
        #break
    new_prices = np.array(prices).astype(np.float)
    return new_prices

In [65]:
# Create Listing Price and Fees
listing_price_fees = host_listings[['id','price', 'weekly_price', 'monthly_price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people']]

show_missing_values(listing_price_fees)

monthly_price       2301
security_deposit    1952
weekly_price        1809
cleaning_fee        1030
dtype: int64

In [66]:
# clean prices
listing_price_fees['price'] = convert_price(listing_price_fees['price'])
listing_price_fees['weekly_price'] = convert_price(listing_price_fees['weekly_price'])
listing_price_fees['monthly_price'] = convert_price(listing_price_fees['monthly_price'])
listing_price_fees['security_deposit'] = convert_price(listing_price_fees['security_deposit'])
listing_price_fees['cleaning_fee'] = convert_price(listing_price_fees['cleaning_fee'])
listing_price_fees['extra_people'] = convert_price(listing_price_fees['extra_people'])
listing_price_fees['guests_included'] = listing_price_fees['guests_included'].fillna('UNKNOWN')


listing_price_fees = listing_price_fees.rename(columns=
                                               {"id":"listing_id",
                                               "extra_people":"extra_people_fee"
                                               })

listing_price_fees.head()

,listing_id,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people_fee
0,241032,85.0,0.0,0.0,0.0,0.0,2,5.0
1,953595,150.0,1000.0,3000.0,100.0,40.0,1,0.0
2,3308979,975.0,0.0,0.0,1000.0,300.0,10,25.0
3,7421966,100.0,650.0,2300.0,0.0,0.0,1,0.0
4,278830,450.0,0.0,0.0,700.0,125.0,6,15.0


In [67]:
listing_price_fees.describe()

,listing_id,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people_fee
count,3.818000e+03,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000
mean,5.550111e+06,127.976166,414.892614,1038.352803,140.566003,45.062860,1.672603,10.667627
std,2.962660e+06,90.250022,551.477118,1677.470398,237.387534,49.913798,1.311040,17.585922
min,3.335000e+03,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.258256e+06,75.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,6.118244e+06,100.000000,262.000000,0.000000,0.000000,30.000000,1.000000,0.000000
75%,8.035127e+06,150.000000,671.500000,1890.000000,200.000000,65.000000,2.000000,20.000000
max,1.034016e+07,1000.000000,6300.000000,19500.000000,5000.000000,300.000000,15.000000,300.000000


In [68]:
# add dummy variables 
listing_price_fees['has_weekly_price'] = [1 if x > 0 else 0 for x in listing_price_fees.weekly_price]
listing_price_fees['has_monthly_price'] = [1 if x > 0 else 0 for x in listing_price_fees.monthly_price]
listing_price_fees['has_security_deposit'] = [1 if x > 0 else 0 for x in listing_price_fees.security_deposit]
listing_price_fees['has_cleaning_fee'] = [1 if x > 0 else 0 for x in listing_price_fees.cleaning_fee]
listing_price_fees['has_extra_people_fee'] = [1 if x > 0 else 0 for x in listing_price_fees.extra_people_fee]

In [69]:
# calculate weekly and monthly discount rates
listing_weekly_price_discounts = []
listing_weekly_price_discount_rates = []
listings_montly_price_discounts = []
listings_montly_price_discount_rates = []

for i,row in listing_price_fees.iterrows():
    
    if row['weekly_price'] > 0:
        listing_weekly_price_discounts.append(row['price']*7-row['weekly_price'])
        listing_weekly_price_discount_rates.append( round (1-(row['weekly_price']/(row['price']*7)),4))
    else:
        listing_weekly_price_discounts.append(0)
        listing_weekly_price_discount_rates.append(0.0)
        
    if row['monthly_price'] > 0:
        listings_montly_price_discounts.append(row['price']*30-row['monthly_price'])
        listings_montly_price_discount_rates.append(round(1-(row['monthly_price']/(row['price']*30)),4))
    else:
        listings_montly_price_discounts.append(0)
        listings_montly_price_discount_rates.append(0)
    
        
listing_price_fees['weekly_discount_offer'] = listing_weekly_price_discounts
listing_price_fees['monthly_discount_offer'] = listings_montly_price_discounts
listing_price_fees['weekly_discount_rate'] = listing_weekly_price_discount_rates
listing_price_fees['monthly_discount_rate'] = listings_montly_price_discount_rates

listing_price_fees.head()

,listing_id,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people_fee,has_weekly_price,has_monthly_price,has_security_deposit,has_cleaning_fee,has_extra_people_fee,weekly_discount_offer,monthly_discount_offer,weekly_discount_rate,monthly_discount_rate
0,241032,85.0,0.0,0.0,0.0,0.0,2,5.0,0,0,0,0,1,0.0,0.0,0.0000,0.0000
1,953595,150.0,1000.0,3000.0,100.0,40.0,1,0.0,1,1,1,1,0,50.0,1500.0,0.0476,0.3333
2,3308979,975.0,0.0,0.0,1000.0,300.0,10,25.0,0,0,1,1,1,0.0,0.0,0.0000,0.0000
3,7421966,100.0,650.0,2300.0,0.0,0.0,1,0.0,1,1,0,0,0,50.0,700.0,0.0714,0.2333
4,278830,450.0,0.0,0.0,700.0,125.0,6,15.0,0,0,1,1,1,0.0,0.0,0.0000,0.0000


In [70]:
listing_price_fees2 = listing_price_fees.merge(listing_calendar_available,how='inner',on='listing_id', copy=True)
listing_price_fees2.head()

,listing_id,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people_fee,has_weekly_price,has_monthly_price,has_security_deposit,has_cleaning_fee,has_extra_people_fee,weekly_discount_offer,monthly_discount_offer,weekly_discount_rate,monthly_discount_rate,calendar_available_days,avg_calendar_price
0,241032,85.0,0.0,0.0,0.0,0.0,2,5.0,0,0,0,0,1,0.0,0.0,0.0000,0.0000,346,85.000000
1,953595,150.0,1000.0,3000.0,100.0,40.0,1,0.0,1,1,1,1,0,50.0,1500.0,0.0476,0.3333,291,170.931272
2,3308979,975.0,0.0,0.0,1000.0,300.0,10,25.0,0,0,1,1,1,0.0,0.0,0.0000,0.0000,220,1301.818182
3,7421966,100.0,650.0,2300.0,0.0,0.0,1,0.0,1,1,0,0,0,50.0,700.0,0.0714,0.2333,143,100.000000
4,278830,450.0,0.0,0.0,700.0,125.0,6,15.0,0,0,1,1,1,0.0,0.0,0.0000,0.0000,365,462.739726


In [71]:
#occupancy_atts = host_listings[['id','minimum_nights','reviews_per_month','number_of_reviews']]
#occupancy_atts = occupancy_atts.fillna(0)
#occupancy_atts = occupancy_atts.rename(columns={"id":"listing_id"})

#listing_price_fees2 = listing_price_fees2.merge(occupancy_atts,how='inner',on='listing_id', copy=True)

listing_price_fees2.to_csv(f'{cleanPath}/listing_price_fees2.csv', index=False)
listing_price_fees2.head()

,listing_id,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people_fee,has_weekly_price,has_monthly_price,has_security_deposit,has_cleaning_fee,has_extra_people_fee,weekly_discount_offer,monthly_discount_offer,weekly_discount_rate,monthly_discount_rate,calendar_available_days,avg_calendar_price
0,241032,85.0,0.0,0.0,0.0,0.0,2,5.0,0,0,0,0,1,0.0,0.0,0.0000,0.0000,346,85.000000
1,953595,150.0,1000.0,3000.0,100.0,40.0,1,0.0,1,1,1,1,0,50.0,1500.0,0.0476,0.3333,291,170.931272
2,3308979,975.0,0.0,0.0,1000.0,300.0,10,25.0,0,0,1,1,1,0.0,0.0,0.0000,0.0000,220,1301.818182
3,7421966,100.0,650.0,2300.0,0.0,0.0,1,0.0,1,1,0,0,0,50.0,700.0,0.0714,0.2333,143,100.000000
4,278830,450.0,0.0,0.0,700.0,125.0,6,15.0,0,0,1,1,1,0.0,0.0,0.0000,0.0000,365,462.739726


In [72]:
temp_reviews2 = pd.read_csv(f'{cleanPath}/listing_review_scores2.csv', error_bad_lines=False, encoding="ISO-8859-1")
#temp_reviews2.head()

temp_occ = temp_reviews2[['listing_id','minimum_nights','number_of_reviews','reviews_per_month','review_days_range','review_years_range']]

listing_price_occ = listing_price_fees2.merge(temp_occ,how='inner',on='listing_id', copy=True)
listing_price_occ.head()

,listing_id,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people_fee,has_weekly_price,has_monthly_price,has_security_deposit,has_cleaning_fee,has_extra_people_fee,weekly_discount_offer,monthly_discount_offer,weekly_discount_rate,monthly_discount_rate,calendar_available_days,avg_calendar_price,minimum_nights,number_of_reviews,reviews_per_month,review_days_range,review_years_range
0,241032,85.0,0.0,0.0,0.0,0.0,2,5.0,0,0,0,0,1,0.0,0.0,0.0000,0.0000,346,85.000000,1,207,4.07,1524,4.175342
1,953595,150.0,1000.0,3000.0,100.0,40.0,1,0.0,1,1,1,1,0,50.0,1500.0,0.0476,0.3333,291,170.931272,2,43,1.48,863,2.364384
2,3308979,975.0,0.0,0.0,1000.0,300.0,10,25.0,0,0,1,1,1,0.0,0.0,0.0000,0.0000,220,1301.818182,4,20,1.15,401,1.098630
3,278830,450.0,0.0,0.0,700.0,125.0,6,15.0,0,0,1,1,1,0.0,0.0,0.0000,0.0000,365,462.739726,1,38,0.89,1202,3.293151
4,5956968,120.0,800.0,0.0,0.0,40.0,1,30.0,1,0,0,1,1,40.0,0.0,0.0476,0.0000,302,125.629139,1,17,2.45,110,0.301370


In [73]:
#temp_reviews2.review_years_range.describe()
#listing_price_fees2.minimum_nights.describe()

In [74]:
temp_occ_df = pd.DataFrame()
temp_occ_df['listing_id'] = listing_price_occ['listing_id']
temp_occ_df['minimum_nights'] = listing_price_occ['minimum_nights']
est_lifetime_occupancy = None
est_lifetime_occupancy_daily_rate = None
est_lifetime_occupancy_yearly_rate = None
est_lifetime_income = None
est_lifetime_yearly_income = None

# estimate occupancy rate
est_lifetime_occupancy = listing_price_occ.minimum_nights*listing_price_occ.number_of_reviews #number of reviews spans multiple years
#est_occupancy_monthly = est_occupancy_yearly/12

# lifetime occupancy estimate
est_lifetime_occupancy_daily_rate = est_lifetime_occupancy/listing_price_occ.review_days_range

# est_occupancy_lifetime_yearly_rate
est_lifetime_occupancy_yearly_rate = est_lifetime_occupancy/listing_price_occ.review_years_range

# estimated lifetime income
est_lifetime_income = est_lifetime_occupancy*listing_price_occ.price

# estimated lifetime yearly income
est_lifetime_yearly_income = est_lifetime_occupancy_yearly_rate*listing_price_occ.price

# estimated lifetime potential income - calendar days available for the year * the number of years * price
est_lifetime_potential_income = (listing_price_occ.calendar_available_days*listing_price_occ.review_years_range)*listing_price_occ.price

# estimated lifetime potential yearly
est_lifetime_potential_yearly_income = est_lifetime_potential_income/listing_price_occ.review_years_range

# percent of potential income estimates
est_perc_yearly_income_of_potential = (est_lifetime_yearly_income/est_lifetime_potential_yearly_income)*100

#est_perc_lifetime_income_potential = (est_lifetime_income/est_lifetime_potential_income)*100

temp_occ_df['est_lifetime_occ'] = est_lifetime_occupancy
temp_occ_df['est_lifetime_occ_daily_rate'] = est_lifetime_occupancy_daily_rate
temp_occ_df['est_lifetime_occ_yearly_rate'] = est_lifetime_occupancy_yearly_rate
temp_occ_df['est_lifetime_income'] = est_lifetime_income
temp_occ_df['est_lifetime_yearly_income'] = est_lifetime_yearly_income
temp_occ_df['est_lifetime_potential_income'] = est_lifetime_potential_income
temp_occ_df['est_lifetime_potential_yearly_income'] = est_lifetime_potential_yearly_income
temp_occ_df['est_perc_yearly_income_of_potential'] = est_perc_yearly_income_of_potential
#temp_occ_df['est_perc_lifetime_income_potential'] = est_perc_lifetime_income_potential

temp_occ_df.head()





,listing_id,minimum_nights,est_lifetime_occ,est_lifetime_occ_daily_rate,est_lifetime_occ_yearly_rate,est_lifetime_income,est_lifetime_yearly_income,est_lifetime_potential_income,est_lifetime_potential_yearly_income,est_perc_yearly_income_of_potential
0,241032,1,207,0.135827,49.576772,17595.0,4214.025591,122796.821918,29410.0,14.328547
1,953595,2,86,0.099652,36.373117,12900.0,5455.967555,103205.342466,43650.0,12.499353
2,3308979,4,80,0.199501,72.817955,78000.0,70997.506234,235656.164384,214500.0,33.099071
3,278830,1,38,0.031614,11.539101,17100.0,5192.595674,540900.000000,164250.0,3.161398
4,5956968,1,17,0.154545,56.409091,2040.0,6769.090909,10921.643836,36240.0,18.678507


In [75]:
listing_price_fees3 = listing_price_fees2.merge(temp_occ_df,how='inner',on='listing_id', copy=True)
listing_price_fees3.to_csv(f'{cleanPath}/listing_price_fees3.csv', index=False)
listing_price_fees3.head()

,listing_id,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people_fee,has_weekly_price,has_monthly_price,has_security_deposit,has_cleaning_fee,has_extra_people_fee,weekly_discount_offer,monthly_discount_offer,weekly_discount_rate,monthly_discount_rate,calendar_available_days,avg_calendar_price,minimum_nights,est_lifetime_occ,est_lifetime_occ_daily_rate,est_lifetime_occ_yearly_rate,est_lifetime_income,est_lifetime_yearly_income,est_lifetime_potential_income,est_lifetime_potential_yearly_income,est_perc_yearly_income_of_potential
0,241032,85.0,0.0,0.0,0.0,0.0,2,5.0,0,0,0,0,1,0.0,0.0,0.0000,0.0000,346,85.000000,1,207,0.135827,49.576772,17595.0,4214.025591,122796.821918,29410.0,14.328547
1,953595,150.0,1000.0,3000.0,100.0,40.0,1,0.0,1,1,1,1,0,50.0,1500.0,0.0476,0.3333,291,170.931272,2,86,0.099652,36.373117,12900.0,5455.967555,103205.342466,43650.0,12.499353
2,3308979,975.0,0.0,0.0,1000.0,300.0,10,25.0,0,0,1,1,1,0.0,0.0,0.0000,0.0000,220,1301.818182,4,80,0.199501,72.817955,78000.0,70997.506234,235656.164384,214500.0,33.099071
3,278830,450.0,0.0,0.0,700.0,125.0,6,15.0,0,0,1,1,1,0.0,0.0,0.0000,0.0000,365,462.739726,1,38,0.031614,11.539101,17100.0,5192.595674,540900.000000,164250.0,3.161398
4,5956968,120.0,800.0,0.0,0.0,40.0,1,30.0,1,0,0,1,1,40.0,0.0,0.0476,0.0000,302,125.629139,1,17,0.154545,56.409091,2040.0,6769.090909,10921.643836,36240.0,18.678507


In [76]:
#high = temp_occ_df['est_perc_yearly_income_of_potential'].quantile(.75)
#qs = pd.qcut(temp_occ_df['est_perc_yearly_income_of_potential'], 3, labels=False)
#temp_occ_df = temp_occ_df.assign(quantile=qs.values)
#temp_occ_df.head(50)

In [77]:
#est_lifetime_occupancy.describe()
#est_lifetime_occupancy_daily_rate.describe()
#est_lifetime_occupancy_yearly_rate.describe()
#est_lifetime_income.describe()
est_lifetime_yearly_income.describe()

count    3.121000e+03
mean     2.113325e+04
std      6.949468e+04
min      1.095000e+02
25%      3.007952e+03
50%      6.090156e+03
75%      1.274878e+04
max      1.531469e+06
dtype: float64

In [78]:
#listing_price_fees3.est_income_of_potential.describe()
#np.max(np.array(listing_price_fees3['est_income_of_potential']))

In [79]:
temp_occ_df.to_csv(f'{cleanPath}/estimated_occupancy.csv', index=False)

# Listing Availability

In [80]:
# Create Listing Availability
listing_availability = host_listings[['id','calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped']]

show_missing_values(listing_availability)

Series([], dtype: int64)

In [81]:

listing_availability['has_availability'] = listing_availability['has_availability'].fillna('f')
listing_availability['availability_30'] = listing_availability['availability_30'].fillna(0)
listing_availability['availability_60'] = listing_availability['availability_60'].fillna(0)
listing_availability['availability_90'] = listing_availability['availability_90'].fillna(0)
listing_availability['availability_365'] = listing_availability['availability_365'].fillna(0)


listing_availability = listing_availability.rename(columns=
                                               {"id":"listing_id",
                                               
                                               })

listing_availability.head()

,listing_id,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped
0,241032,4 weeks ago,t,14,41,71,346,2016-01-04
1,953595,today,t,13,13,16,291,2016-01-04
2,3308979,5 weeks ago,t,1,6,17,220,2016-01-04
3,7421966,6 months ago,t,0,0,0,143,2016-01-04
4,278830,7 weeks ago,t,30,60,90,365,2016-01-04


# Listing Review Scores

In [82]:
# Create Listing Review Scores
listing_review_scores = host_listings[['id','number_of_reviews',
       'first_review', 'last_review', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value','reviews_per_month']]

show_missing_values(listing_review_scores)

review_scores_checkin          658
review_scores_accuracy         658
review_scores_value            656
review_scores_location         655
review_scores_cleanliness      653
review_scores_communication    651
review_scores_rating           647
reviews_per_month              627
last_review                    627
first_review                   627
dtype: int64

In [83]:
np.mean(np.array(listing_review_scores['review_scores_rating']), dtype=np.float64)

nan

In [84]:
# clean dataset - fill nan values with mean

listing_review_scores['first_review'] = listing_review_scores['first_review'].fillna('1900-01-01')
listing_review_scores['last_review'] = listing_review_scores['last_review'].fillna('1900-01-01')
listing_review_scores['review_scores_rating'] = listing_review_scores['review_scores_rating'].fillna(np.nanmean(np.array(listing_review_scores['review_scores_rating']), dtype=np.float64))
listing_review_scores['review_scores_accuracy'] = listing_review_scores['review_scores_accuracy'].fillna(np.nanmean(np.array(listing_review_scores['review_scores_accuracy']), dtype=np.float64))
listing_review_scores['review_scores_cleanliness'] = listing_review_scores['review_scores_cleanliness'].fillna(np.nanmean(np.array(listing_review_scores['review_scores_cleanliness']), dtype=np.float64))
listing_review_scores['review_scores_checkin'] = listing_review_scores['review_scores_checkin'].fillna(np.nanmean(np.array(listing_review_scores['review_scores_checkin']), dtype=np.float64))
listing_review_scores['review_scores_communication'] = listing_review_scores['review_scores_communication'].fillna(np.nanmean(np.array(listing_review_scores['review_scores_communication']), dtype=np.float64))
listing_review_scores['review_scores_location'] = listing_review_scores['review_scores_location'].fillna(np.nanmean(np.array(listing_review_scores['review_scores_location']), dtype=np.float64))
listing_review_scores['review_scores_value'] = listing_review_scores['review_scores_value'].fillna(np.nanmean(np.array(listing_review_scores['review_scores_value']), dtype=np.float64))
listing_review_scores['reviews_per_month'] = listing_review_scores['reviews_per_month'].fillna(np.nanmean(np.array(listing_review_scores['reviews_per_month']), dtype=np.float64))


listing_review_scores = listing_review_scores.rename(columns=
                                               {"id":"listing_id",
                                               
                                               })

listing_review_scores.head()

,listing_id,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,241032,207,2011-11-01,2016-01-02,95.000000,10.000000,10.000000,10.000000,10.000000,9.000000,10.000000,4.070000
1,953595,43,2013-08-19,2015-12-29,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.480000
2,3308979,20,2014-07-30,2015-09-03,97.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.150000
3,7421966,0,1900-01-01,1900-01-01,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,2.078919
4,278830,38,2012-07-10,2015-10-24,92.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,0.890000


In [85]:
# A new feature creations. new_score_reviews creations from number of score x frequency feature.
listing_review_scores['new_score_reviews'] = listing_review_scores['reviews_per_month'] * listing_review_scores['review_scores_rating'] / 10
listing_review_scores['new_score_reviews'].fillna(0,inplace=True)
listing_review_scores.head()

,listing_id,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,new_score_reviews
0,241032,207,2011-11-01,2016-01-02,95.000000,10.000000,10.000000,10.000000,10.000000,9.000000,10.000000,4.070000,38.665000
1,953595,43,2013-08-19,2015-12-29,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.480000,14.208000
2,3308979,20,2014-07-30,2015-09-03,97.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.150000,11.155000
3,7421966,0,1900-01-01,1900-01-01,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,2.078919,19.653945
4,278830,38,2012-07-10,2015-10-24,92.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,0.890000,8.188000


Decide the threshold of Top Performer and Low Performer listings. I defined the first threshold based on the 1st quartile of new_score_review so the listings that score less than the 1st threshold will be classified as the low performer and for the classification of top performer I defined whom listings' score more than 90% quartile.

In [86]:
'''
the definition and print the value.
'''
top90flag = listing_review_scores['new_score_reviews'].quantile(0.9)
upto25flag = listing_review_scores['new_score_reviews'].quantile(0.25)

listing_review_scores['top90'] = listing_review_scores['new_score_reviews'] >= top90flag
listing_review_scores['upto25'] = listing_review_scores['new_score_reviews'] <= upto25flag

print('The boundaries of top performer listings:',top90flag)
print('The boundaries of low performer listings:',upto25flag)

The boundaries of top performer listings: 41.27720000000001
The boundaries of low performer listings: 7.88875


In [87]:
listing_review_scores['upto25'][:5]

0    False
1    False
2    False
3    False
4    False
Name: upto25, dtype: bool

In [88]:
### Create a table for the visualization essentials ###
### Generate score bins, creating new tables for the class colors, and count distributions of each bins. ###
# a columns of bins.
listing_review_scores['score_ranges'] = listing_review_scores['new_score_reviews'].apply(rangeScore)

# table coloring purpose.
top90 = listing_review_scores.groupby('score_ranges', as_index = False)['top90'].max(key = 'count').rename(columns={'score_ranges':'Score'})
upto25 = listing_review_scores.groupby('score_ranges', as_index = False)['upto25'].max(key = 'count').rename(columns={'score_ranges':'Score'})

# count distributions of score bins.
barplot = listing_review_scores[['listing_id','new_score_reviews']]
barplot['Qty'] = barplot['new_score_reviews'].apply(rangeScore)
barplot = barplot.Qty.value_counts(sort=True).to_frame().reset_index()
barplot = barplot.rename(columns={'index': 'Score'})

# merging color flag.
barplot = barplot.merge(top90, on = 'Score')
barplot = barplot.merge(upto25)
# creating color for the vis.
barplot['color'] = barplot.top90.apply(lambda x: 'mediumaquamarine' if x else 'lightgray')
# manually change the color of the first index become crimson, to indicate the class of low performer listings.
barplot.iloc[0,4] = 'crimson'

# change Score column and Qty column into x and y for the vis purpose.
barplot = barplot.rename(columns={'Score':'x','Qty':'y'})

# Some of the annotations for the vis.
title_text = ['<b>Comparison Listings Performance between Top Performer and Low Performer</b>', 'Reviews per Month x Review Score Ratings / 10', 'Quantity of Listings']
annotations = [{'x': 0.03, 'y': 1900, 'text': 'Low Performer Had Score Up to 25 Percentile','color': 'gray'},
              {'x': 0.39, 'y': 300, 'text': 'Top Performer Had Score above 90 Percentile','color': 'mediumaquamarine'}]

generate_barplot(title_text, annotations)

On the bar chat above the red bar one is the low performer it represents under percentile 0.25 of the score and the aquamarine one is the high performer it has score more than 90 percentile of the distributions.

In [89]:
listing_review_scores['top90'] = [1 if x == 'True' else 0 for x in listing_review_scores['top90']]
listing_review_scores['upto25'] = [1 if x == 'True' else 0 for x in listing_review_scores['upto25']]

In [90]:
# merge sentiment average review scores with listing_review_scores
listing_reviews_avg_sentiment_scores = pd.read_csv(f'{cleanPath}/listing_reviews_avg_sentiment_scores.csv', error_bad_lines=False, encoding="ISO-8859-1")

listing_reviews_avg_sentiment_scores.head()

,listing_id,avg_pol_pos_score,avg_pol_neg_score,avg_pol_neu_score,avg_pol_compound_score,avg_review_sentiment
0,9367465,0.224,0.024,0.752,0.9965,positive
1,3888986,0.394,0.000,0.606,0.9960,positive
2,5968862,0.305,0.017,0.678,0.9958,positive
3,8988178,0.318,0.011,0.672,0.9947,positive
4,8483744,0.207,0.010,0.783,0.9947,positive


In [91]:
dummies = pd.get_dummies(listing_reviews_avg_sentiment_scores['avg_review_sentiment'], prefix = 'avg_review_sentiment')
listing_reviews_avg_sentiment_scores = pd.concat([listing_reviews_avg_sentiment_scores,dummies], axis = 1)

listing_reviews_avg_sentiment_scores.head()

,listing_id,avg_pol_pos_score,avg_pol_neg_score,avg_pol_neu_score,avg_pol_compound_score,avg_review_sentiment,avg_review_sentiment_negative,avg_review_sentiment_neutral,avg_review_sentiment_positive
0,9367465,0.224,0.024,0.752,0.9965,positive,0,0,1
1,3888986,0.394,0.000,0.606,0.9960,positive,0,0,1
2,5968862,0.305,0.017,0.678,0.9958,positive,0,0,1
3,8988178,0.318,0.011,0.672,0.9947,positive,0,0,1
4,8483744,0.207,0.010,0.783,0.9947,positive,0,0,1


In [92]:
listing_review_scores2 = listing_review_scores.merge(listing_reviews_avg_sentiment_scores,how='inner',on='listing_id', copy=True)
listing_review_scores2.head()

,listing_id,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,new_score_reviews,top90,upto25,score_ranges,avg_pol_pos_score,avg_pol_neg_score,avg_pol_neu_score,avg_pol_compound_score,avg_review_sentiment,avg_review_sentiment_negative,avg_review_sentiment_neutral,avg_review_sentiment_positive
0,241032,207,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,4.07,38.665,0,0,30-40,0.2949,0.0186,0.6865,0.8840,positive,0,0,1
1,953595,43,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,1.48,14.208,0,0,10-20,0.2858,0.0131,0.7012,0.8967,positive,0,0,1
2,3308979,20,2014-07-30,2015-09-03,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.15,11.155,0,0,10-20,0.3169,0.0136,0.6694,0.8858,positive,0,0,1
3,278830,38,2012-07-10,2015-10-24,92.0,9.0,9.0,10.0,10.0,9.0,9.0,0.89,8.188,0,0,0-10,0.2958,0.0051,0.6991,0.8368,positive,0,0,1
4,5956968,17,2015-06-11,2015-09-28,95.0,10.0,10.0,10.0,10.0,10.0,10.0,2.45,23.275,0,0,20-30,0.2865,0.0075,0.7059,0.9486,positive,0,0,1


In [93]:
temp_min_nights = host_listings[['id','minimum_nights']]
temp_min_nights = temp_min_nights.rename(columns={"id":"listing_id"})

listing_review_scores2 = listing_review_scores2.merge(temp_min_nights, how='inner', on='listing_id', copy=True)                                        

In [94]:
first_review_dates = pd.to_datetime(listing_review_scores2['first_review'])
last_review_dates = pd.to_datetime(listing_review_scores2['last_review'])
review_days_range = ((last_review_dates - first_review_dates).dt.days)+1
review_frequency = listing_review_scores2['number_of_reviews']*listing_review_scores2['minimum_nights']/review_days_range
#review_days_range[:5]
#review_frequency.describe()

listing_review_scores2['review_days_range'] = review_days_range
listing_review_scores2['review_years_range'] = review_days_range/365
listing_review_scores2['reviews_per_year_rate'] = listing_review_scores2['number_of_reviews']/listing_review_scores2['review_years_range']
listing_review_scores2['review_frequency'] = review_frequency


#listing_review_scores.head()

In [95]:
listing_review_scores.to_csv(f'{cleanPath}/listing_review_scores.csv', index=False)
listing_review_scores2.to_csv(f'{cleanPath}/listing_review_scores2.csv', index=False)

# Listing Policies

In [96]:
# Create Listing Policies
listing_policies = host_listings[['id','minimum_nights',
       'maximum_nights','cancellation_policy','require_guest_profile_picture',
       'require_guest_phone_verification','requires_license',
       'license','instant_bookable']]

show_missing_values(listing_policies)

license    3818
dtype: int64

In [97]:
#t = [convert_to_binary(x) for x in listing_policies['instant_bookable']]

t = list(listing_policies['instant_bookable'])
t.index('t')

14

In [98]:
# Create Listing Policies


listing_policies['license'] = listing_policies['license'].fillna('UNKNOWN')
listing_policies['cancellation_policy'] = listing_policies['cancellation_policy'].fillna('UNKNOWN')
listing_policies['minimum_nights'] = listing_policies['minimum_nights'].fillna(0)
listing_policies['maximum_nights'] = listing_policies['maximum_nights'].fillna(0)
listing_policies['requires_license'] = [1 if x == 't' else 0 for x in listing_policies['requires_license']]
listing_policies['requires_license'] = listing_policies['requires_license'].fillna(0)
listing_policies['require_guest_phone_verification'] = [1 if x == 't' else 0 for x in listing_policies['require_guest_phone_verification']]
listing_policies['require_guest_phone_verification'] = listing_policies['require_guest_phone_verification'].fillna(0)
listing_policies['require_guest_profile_picture'] = [1 if x == 't' else 0 for x in listing_policies['require_guest_profile_picture']]
listing_policies['require_guest_profile_picture'] = listing_policies['require_guest_profile_picture'].fillna(0)
listing_policies['instant_bookable'] = [convert_to_binary(str(x).strip()) for x in listing_policies['instant_bookable']]
listing_policies['instant_bookable'] = listing_policies['instant_bookable'].fillna(0)


listing_policies = listing_policies.rename(columns=
                                               {"id":"listing_id",
                                               
                                               })
listing_policies.to_csv(f'{cleanPath}/listing_policies.csv', index=False)
listing_policies.head()

,listing_id,minimum_nights,maximum_nights,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,requires_license,license,instant_bookable
0,241032,1,365,moderate,0,0,0,UNKNOWN,0
1,953595,2,90,strict,1,1,0,UNKNOWN,0
2,3308979,4,30,strict,0,0,0,UNKNOWN,0
3,7421966,1,1125,flexible,0,0,0,UNKNOWN,0
4,278830,1,1125,strict,0,0,0,UNKNOWN,0


# Listing to Host Mapping

In [99]:
listings_host_map = host_listings[['id','host_id','listing_url','scrape_id','last_scraped','name']]

listings_host_map['listing_url'] = listings_host_map['listing_url'].fillna('UNKNOWN')
listings_host_map['name'] = listings_host_map['name'].fillna('UNKNOWN')


listings_host_map = listings_host_map.rename(columns=
                                               {"id":"listing_id",
                                               "name":"listing_name"
                                               })

listings_host_map.head()

,listing_id,host_id,listing_url,scrape_id,last_scraped,listing_name
0,241032,956883,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment
1,953595,5177328,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment
2,3308979,16708587,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view
3,7421966,9851441,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau
4,278830,1452570,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house


# Host Listing Counts

In [100]:
host_listings_counts = host_listings[['id','host_id','host_listings_count', 'host_total_listings_count','calculated_host_listings_count']]


host_listings_counts['host_listings_count'] = host_listings_counts['host_listings_count'].fillna(-1)
host_listings_counts['host_total_listings_count'] = host_listings_counts['host_total_listings_count'].fillna(-1)
host_listings_counts['calculated_host_listings_count'] = host_listings_counts['calculated_host_listings_count'].fillna(-1)

host_listings_counts = host_listings_counts.rename(columns=
                                               {"id":"listing_id",
                                               
                                               })

host_listings_counts.head()

,listing_id,host_id,host_listings_count,host_total_listings_count,calculated_host_listings_count
0,241032,956883,3.0,3.0,2
1,953595,5177328,6.0,6.0,6
2,3308979,16708587,2.0,2.0,2
3,7421966,9851441,1.0,1.0,1
4,278830,1452570,2.0,2.0,1


# Host Attributes

In [101]:
def changeTime(x):
    '''
    change host_response_time columns from string into numerical.
    '''
    if x == 'within an hour':
        x='1'
    elif x == 'within a few hours':
        x='4'
    elif x == 'within a day':
        x='24'
    elif x == 'a few days or more':
        x='48'
    else:
        x='96'
        
    return x

In [105]:
# Create Host Object
#host_ids = []
#hosts = []
#listing_hosts = host_listings[['id','host_id', 'host_url', 'host_name', 'host_since',
       #'host_location', 'host_about', 'host_response_time',
       #'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       #'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_total_listings_count',
       #'host_verifications', 'host_has_profile_pic', 'host_identity_verified']]

#for i, row in listing_hosts.iterrows():
#    host_id = row['host_id']
#    if host_id not in host_ids:
#        host_ids.append(host_id)
#        hosts.append(row)
        
#hosts[:5]
#hosts_df = pd.DataFrame(data=hosts)
#hosts_df = hosts_df.sort_values(by=['host_id']).head()

In [106]:
#hosts_df.shape

In [107]:
listing_hosts = host_listings[['id','host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified']]

In [108]:
#listing_hosts = hosts_df

listing_hosts['host_url'] = listing_hosts['host_url'].fillna('UNKNOWN')
listing_hosts['host_name'] = listing_hosts['host_name'].fillna('UNKNOWN')
listing_hosts['host_since'] = listing_hosts['host_since'].fillna('1900-01-01')
listing_hosts['host_location'] = listing_hosts['host_location'].fillna('UNKNOWN')
listing_hosts['host_about'] = listing_hosts['host_url'].fillna('host_about')
listing_hosts['host_response_time'] = listing_hosts['host_response_time'].fillna(0)
listing_hosts['host_response_rate'] = listing_hosts['host_response_rate'].fillna(0)
listing_hosts['host_acceptance_rate'] = listing_hosts['host_acceptance_rate'].fillna(0)
listing_hosts['host_is_superhost'] = [1 if x == 't' else 0 for x in listing_hosts['host_is_superhost']]
listing_hosts['host_is_superhost'] = listing_hosts['host_is_superhost'].fillna(0)
listing_hosts['host_thumbnail_url'] = listing_hosts['host_thumbnail_url'].fillna('UNKNOWN')
listing_hosts['host_picture_url'] = listing_hosts['host_picture_url'].fillna('UNKNOWN')
listing_hosts['host_neighbourhood'] = listing_hosts['host_neighbourhood'].fillna('UNKNOWN')
#listing_hosts['host_listings_count'] = listing_hosts['host_listings_count'].fillna(0)
listing_hosts['host_total_listings_count'] = listing_hosts['host_total_listings_count'].fillna(0)
listing_hosts['host_verifications'] = listing_hosts['host_verifications'].fillna('UNKNOWN')
listing_hosts['host_has_profile_pic'] = [1 if x == 't' else 0 for x in listing_hosts['host_has_profile_pic']]
listing_hosts['host_has_profile_pic'] = listing_hosts['host_has_profile_pic'].fillna(0)
listing_hosts['host_identity_verified'] = [1 if x == 't' else 0 for x in listing_hosts['host_identity_verified']]
listing_hosts['host_identity_verified'] = listing_hosts['host_identity_verified'].fillna(0)


listing_hosts = listing_hosts.rename(columns=
                                               {"id":"listing_id",
                                               
                                               })
#listing_hosts = listing_hosts.drop(columns=['listing_id'])
listing_hosts.head()

,listing_id,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified
0,241032,956883,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"Seattle, Washington, United States",https://www.airbnb.com/users/show/956883,within a few hours,96%,100%,0,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,"['email', 'phone', 'reviews', 'kba']",1,1
1,953595,5177328,https://www.airbnb.com/users/show/5177328,Andrea,2013-02-21,"Seattle, Washington, United States",https://www.airbnb.com/users/show/5177328,within an hour,98%,100%,1,https://a0.muscache.com/ac/users/5177328/profi...,https://a0.muscache.com/ac/users/5177328/profi...,Queen Anne,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",1,1
2,3308979,16708587,https://www.airbnb.com/users/show/16708587,Jill,2014-06-12,"Seattle, Washington, United States",https://www.airbnb.com/users/show/16708587,within a few hours,67%,100%,0,https://a1.muscache.com/ac/users/16708587/prof...,https://a1.muscache.com/ac/users/16708587/prof...,Queen Anne,2.0,"['email', 'phone', 'google', 'reviews', 'jumio']",1,1
3,7421966,9851441,https://www.airbnb.com/users/show/9851441,Emily,2013-11-06,"Seattle, Washington, United States",https://www.airbnb.com/users/show/9851441,0,0,0,0,https://a2.muscache.com/ac/users/9851441/profi...,https://a2.muscache.com/ac/users/9851441/profi...,Queen Anne,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",1,1
4,278830,1452570,https://www.airbnb.com/users/show/1452570,Emily,2011-11-29,"Seattle, Washington, United States",https://www.airbnb.com/users/show/1452570,within an hour,100%,0,0,https://a0.muscache.com/ac/users/1452570/profi...,https://a0.muscache.com/ac/users/1452570/profi...,Queen Anne,2.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",1,1


In [109]:
#listing_hosts['host_neighbourhood'].unique()

In [110]:
#listing_hosts['host_response_time'].unique()
host_response_within_an_hour = []
host_response_within_a_few_hours = []
host_response_time_hrs = []

#listing_hosts[listing_hosts['host_response_time'] == 'within a few hours']

for e in listing_hosts['host_response_time']:
    if e == 'within an hour':
        host_response_within_an_hour.append(1)
    else:
        host_response_within_an_hour.append(0)

    if e == 'within a few hours':
        host_response_within_a_few_hours.append(1)
    else:
        host_response_within_a_few_hours.append(0)
        
listing_hosts['host_response_within_an_hour'] = host_response_within_an_hour
listing_hosts['host_response_within_a_few_hours'] = host_response_within_a_few_hours
listing_hosts['host_response_time_hrs'] = [changeTime(t) for t in listing_hosts['host_response_time']]



listing_hosts.to_csv(f'{cleanPath}/listing_hosts.csv', index=False)
listing_hosts.head()

,listing_id,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,host_response_within_an_hour,host_response_within_a_few_hours,host_response_time_hrs
0,241032,956883,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"Seattle, Washington, United States",https://www.airbnb.com/users/show/956883,within a few hours,96%,100%,0,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,"['email', 'phone', 'reviews', 'kba']",1,1,0,1,4
1,953595,5177328,https://www.airbnb.com/users/show/5177328,Andrea,2013-02-21,"Seattle, Washington, United States",https://www.airbnb.com/users/show/5177328,within an hour,98%,100%,1,https://a0.muscache.com/ac/users/5177328/profi...,https://a0.muscache.com/ac/users/5177328/profi...,Queen Anne,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",1,1,1,0,1
2,3308979,16708587,https://www.airbnb.com/users/show/16708587,Jill,2014-06-12,"Seattle, Washington, United States",https://www.airbnb.com/users/show/16708587,within a few hours,67%,100%,0,https://a1.muscache.com/ac/users/16708587/prof...,https://a1.muscache.com/ac/users/16708587/prof...,Queen Anne,2.0,"['email', 'phone', 'google', 'reviews', 'jumio']",1,1,0,1,4
3,7421966,9851441,https://www.airbnb.com/users/show/9851441,Emily,2013-11-06,"Seattle, Washington, United States",https://www.airbnb.com/users/show/9851441,0,0,0,0,https://a2.muscache.com/ac/users/9851441/profi...,https://a2.muscache.com/ac/users/9851441/profi...,Queen Anne,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",1,1,0,0,96
4,278830,1452570,https://www.airbnb.com/users/show/1452570,Emily,2011-11-29,"Seattle, Washington, United States",https://www.airbnb.com/users/show/1452570,within an hour,100%,0,0,https://a0.muscache.com/ac/users/1452570/profi...,https://a0.muscache.com/ac/users/1452570/profi...,Queen Anne,2.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",1,1,1,0,1


## Output New Data Objects

In [111]:
# output file 
cleanPath = f'{dataDir}/clean'
if not os.path.exists(cleanPath): os.makedirs(cleanPath)
    
# output new cleaned objects
listing_description_text.to_csv(f'{cleanPath}/listing_description_text.csv', index=False)
listing_location.to_csv(f'{cleanPath}/listing_location.csv', index=False)
listing_property_features.to_csv(f'{cleanPath}/listing_property_features.csv', index=False)
listing_price_fees.to_csv(f'{cleanPath}/listing_price_fees.csv', index=False)
listing_availability.to_csv(f'{cleanPath}/listing_availability.csv', index=False)
listing_review_scores.to_csv(f'{cleanPath}/listing_review_scores.csv', index=False)
listing_hosts.to_csv(f'{cleanPath}/listing_hosts.csv', index=False)
listing_policies.to_csv(f'{cleanPath}/listing_policies.csv', index=False)
listings_host_map.to_csv(f'{cleanPath}/listings_host_map.csv', index=False)
host_listings_counts.to_csv(f'{cleanPath}/host_listings_counts.csv', index=False)
listing_property_types.to_csv(f'{cleanPath}/listing_property_types.csv', index=False)

cleaned_amenities_df.to_csv(f'{cleanPath}/listing_property_amenities.csv', index=False)


In [112]:
# join useful attributes 

#listing_property_features
#listing_hosts
#listing_policies
#listing_review_scores2
#temp_occ_df
#listing_price_occ
#temp_dummies2
#listing_description_text

In [113]:
logger.info(f'listing_property_features\n{listing_property_features.columns}')

listing_property_features
Index(['listing_id', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
       'bedroom_share', 'bathroom_share', 'private_room', 'shared_room',
       'entire_home_apt', 'real_bed', 'futon_bed', 'pull_out_sofa_bed',
       'air_bed', 'couch_bed'],
      dtype='object')


In [114]:
keep_feats = ['listing_id','accommodates', 'bathrooms',
       'bedrooms', 'beds','bedroom_share', 'bathroom_share', 'private_room', 'shared_room',
       'entire_home_apt', 'real_bed', 'futon_bed', 'pull_out_sofa_bed',
       'air_bed', 'couch_bed']

feats = listing_property_features[keep_feats]
feats.shape

(3818, 15)

In [115]:
logger.info(f'listing_hosts\n{listing_hosts.columns}')

listing_hosts
Index(['listing_id', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'host_response_within_an_hour', 'host_response_within_a_few_hours',
       'host_response_time_hrs'],
      dtype='object')


In [116]:
keep_hosts = ['listing_id', 'host_id','host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost','host_has_profile_pic', 'host_identity_verified',
       'host_response_within_an_hour', 'host_response_within_a_few_hours',
       'host_response_time_hrs']

hosts = listing_hosts[keep_hosts]
hosts.shape

(3818, 11)

In [117]:
logger.info(f'listing_policies\n{listing_policies.columns}')

listing_policies
Index(['listing_id', 'minimum_nights', 'maximum_nights', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'requires_license', 'license', 'instant_bookable'],
      dtype='object')


In [118]:
keep_policies = ['listing_id', 'minimum_nights', 'maximum_nights','require_guest_profile_picture', 'require_guest_phone_verification',
       'requires_license','instant_bookable']

policies = listing_policies[keep_policies]
policies.shape

(3818, 7)

In [119]:
logger.info(f'listing_review_scores2\n{listing_review_scores2.columns}')

listing_review_scores2
Index(['listing_id', 'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'reviews_per_month', 'new_score_reviews',
       'top90', 'upto25', 'score_ranges', 'avg_pol_pos_score',
       'avg_pol_neg_score', 'avg_pol_neu_score', 'avg_pol_compound_score',
       'avg_review_sentiment', 'avg_review_sentiment_negative',
       'avg_review_sentiment_neutral', 'avg_review_sentiment_positive',
       'minimum_nights', 'review_days_range', 'review_years_range',
       'reviews_per_year_rate', 'review_frequency'],
      dtype='object')


In [120]:
keep_rev_scores = ['listing_id','number_of_reviews','review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location','new_score_reviews',
       'top90', 'upto25', 'score_ranges', 'avg_pol_pos_score',
       'avg_pol_neg_score', 'avg_pol_neu_score', 'avg_pol_compound_score',
       'avg_review_sentiment','avg_review_sentiment_negative',
       'avg_review_sentiment_neutral', 'avg_review_sentiment_positive','review_days_range',
       'review_years_range', 'reviews_per_year_rate', 'review_frequency']

rev_scores = listing_review_scores2[keep_rev_scores]
rev_scores.shape

(3191, 24)

In [121]:
logger.info(f'temp_occ_df\n{temp_occ_df.columns}')

temp_occ_df
Index(['listing_id', 'minimum_nights', 'est_lifetime_occ',
       'est_lifetime_occ_daily_rate', 'est_lifetime_occ_yearly_rate',
       'est_lifetime_income', 'est_lifetime_yearly_income',
       'est_lifetime_potential_income', 'est_lifetime_potential_yearly_income',
       'est_perc_yearly_income_of_potential'],
      dtype='object')


In [122]:
keep_occ = ['listing_id','est_lifetime_occ',
       'est_lifetime_occ_daily_rate', 'est_lifetime_occ_yearly_rate',
       'est_lifetime_income', 'est_lifetime_yearly_income',
       'est_lifetime_potential_income', 'est_lifetime_potential_yearly_income',
       'est_perc_yearly_income_of_potential']

occ = temp_occ_df[keep_occ]
occ.shape

(3121, 9)

In [123]:
logger.info(f'listing_price_occ\n{listing_price_occ.columns}')

listing_price_occ
Index(['listing_id', 'price', 'weekly_price', 'monthly_price',
       'security_deposit', 'cleaning_fee', 'guests_included',
       'extra_people_fee', 'has_weekly_price', 'has_monthly_price',
       'has_security_deposit', 'has_cleaning_fee', 'has_extra_people_fee',
       'weekly_discount_offer', 'monthly_discount_offer',
       'weekly_discount_rate', 'monthly_discount_rate',
       'calendar_available_days', 'avg_calendar_price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month', 'review_days_range',
       'review_years_range'],
      dtype='object')


In [124]:
keep_price = ['listing_id', 'price', 'weekly_price', 'monthly_price',
       'security_deposit', 'cleaning_fee', 'guests_included',
       'extra_people_fee', 'has_weekly_price', 'has_monthly_price',
       'has_security_deposit', 'has_cleaning_fee', 'has_extra_people_fee',
       'weekly_discount_offer', 'monthly_discount_offer',
       'weekly_discount_rate', 'monthly_discount_rate']

price = listing_price_occ[keep_price]
price.shape

(3121, 17)

In [125]:
logger.info(f'temp_dummies2\n{temp_dummies2.columns}')

temp_dummies2
Index(['listing_id', '24hour_checkin', 'access_friendly', 'breakfast',
       'essentials', 'family_kid_friendly', 'free_parking_on_premises', 'gym',
       'hot_tub', 'internet', 'kitchen', 'laundry_facility', 'pet_friendly',
       'pool', 'safety_features', 'security_features', 'smoking_allowed',
       'suitable_for_events', 'tv', 'weather_control', 'host_is_superhost_f',
       'host_is_superhost_t', 'host_identity_verified_f',
       'host_identity_verified_t', 'host_has_profile_pic_f',
       'host_has_profile_pic_t', 'neighbourhood_group_cleansed_Ballard',
       'neighbourhood_group_cleansed_Beacon Hill',
       'neighbourhood_group_cleansed_Capitol Hill',
       'neighbourhood_group_cleansed_Cascade',
       'neighbourhood_group_cleansed_Central Area',
       'neighbourhood_group_cleansed_Delridge',
       'neighbourhood_group_cleansed_Downtown',
       'neighbourhood_group_cleansed_Interbay',
       'neighbourhood_group_cleansed_Lake City',
       'neighbourhoo

In [126]:
keep_dummies = list(temp_dummies2.columns)

dummies = temp_dummies2[keep_dummies]
dummies.shape

(3818, 70)

In [127]:
logger.info(f'listing_description_text\n{listing_description_text.columns}')

listing_description_text
Index(['listing_id', 'listing_name', 'summary', 'space', 'description',
       'experiences_offered', 'neighborhood_overview', 'listing_notes',
       'transit', 'host_about'],
      dtype='object')


In [128]:
keep_desc = ['listing_id','listing_name', 'summary', 'space', 'description',
       'experiences_offered', 'neighborhood_overview', 'listing_notes',
       'transit', 'host_about']

desc = listing_description_text[keep_desc]
desc.shape

(3818, 10)

In [129]:
#feats,hosts,policies,rev_scores,occ,price,desc,dummies

In [130]:
# merge data frames
master = feats.merge(hosts,how='inner',on='listing_id')
master = master.merge(policies,how='inner',on='listing_id')
master = master.merge(rev_scores,how='inner',on='listing_id')
master = master.merge(price,how='inner',on='listing_id')
master = master.merge(occ,how='inner',on='listing_id')
master = master.merge(desc,how='inner',on='listing_id')
master = master.merge(dummies,how='inner',on='listing_id')



In [131]:
drop_indexs = master[master['minimum_nights']>31].index
master.drop(drop_indexs, inplace=True)
#master.info()

qs = pd.qcut(master['est_perc_yearly_income_of_potential'], 3, labels=False)
master = master.assign(income_performance_label=qs.values)

In [132]:
logger.info(f'master shape:\n{master.shape}')
logger.info(f'master columns:\n{master.columns}')

master shape:
(3120, 157)
master columns:
Index(['listing_id', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'bedroom_share', 'bathroom_share', 'private_room', 'shared_room',
       'entire_home_apt',
       ...
       'property_type_Yurt', 'bed_type_Airbed', 'bed_type_Couch',
       'bed_type_Futon', 'bed_type_Pull-out Sofa', 'bed_type_Real Bed',
       'cancellation_policy_flexible', 'cancellation_policy_moderate',
       'cancellation_policy_strict', 'income_performance_label'],
      dtype='object', length=157)


In [133]:
master.to_csv(f'{cleanPath}/master.csv', index=False)

In [134]:
keep_occ = ['listing_id','est_lifetime_occ',
       'est_lifetime_occ_daily_rate', 'est_lifetime_occ_yearly_rate',
       'est_lifetime_income', 'est_lifetime_yearly_income',
       'est_lifetime_potential_income', 'est_lifetime_potential_yearly_income',
       'est_perc_yearly_income_of_potential','income_performance_label']

occ2 = master[master[keep_occ]]

occ2.to_csv(f'{cleanPath}/occ2.csv', index=False)

ValueError: Must pass DataFrame with boolean values only

## Engineer neighbourhood score
Add neighbourhoor scores(walk, transit and bike) after scraping data from https://www.walkscore.com/WA/Seattle . These scores are stored in neighborhood_scores file. The features are joined to the listings dataframe using neighbourhood. These scores help in quantifying the location of the listing as none of the existing features quantify listing.

In [ ]:
neighborhood_scores = pd.read_csv(f'{dataDir}/seattle/neighborhood_scores.csv', error_bad_lines=False, encoding="ISO-8859-1")

In [ ]:
neighborhood_scores.head()

In [ ]:
#neighborhood_scores.Name.unique()

# 3. Visualizations

In [ ]:
'''
    ### VIZ FUNCTIONS ###
    this functions actually using the functions from
    https://www.kaggle.com/andresionek/what-makes-a-kaggler-valuable/notebook
'''

def gen_xaxis(title):
    """
    Creates the X Axis layout and title
    """
    xaxis = dict(
            title=title,
            titlefont=dict(
                color='#AAAAAA'
            ),
            showgrid=False,
            color='#AAAAAA',
            )
    return xaxis


def gen_yaxis(title):
    """
    Creates the Y Axis layout and title
    """
    yaxis=dict(
            title=title,
            titlefont=dict(
                color='#AAAAAA'
            ),
            showgrid=False,
            color='#AAAAAA',
            )
    return yaxis


def gen_layout(charttitle, xtitle, ytitle, lmarg, h, annotations=None):  
    """
    Creates whole layout, with both axis, annotations, size and margin
    """
    return go.Layout(title=charttitle, 
                     height=h, 
                     width=800,
                     showlegend=False,
                     xaxis=gen_xaxis(xtitle), 
                     yaxis=gen_yaxis(ytitle),
                     annotations = annotations,
                     margin=dict(l=lmarg),
                    )


def gen_bars(data, color, orient):
    """
    Generates the bars for plotting, with their color and orient
    """
    bars = []
    for label, label_df in data.groupby(color):
        if orient == 'h':
            label_df = label_df.sort_values(by='x', ascending=True)
        if label == 'a':
            label = 'lightgray'
        bars.append(go.Bar(x=label_df.x,
                           y=label_df.y,
                           name=label,
                           marker={'color': label},
                           orientation = orient
                          )
                   )
    return bars


def gen_annotations(annot):
    """
    Generates annotations to insert in the chart
    """
    if annot is None:
        return []
    
    annotations = []
    # Adding labels
    for d in annot:
        annotations.append(dict(xref='paper', x=d['x'], y=d['y'],
                           xanchor='left', yanchor='bottom',
                           text= d['text'],
                           font=dict(size=13,
                           color=d['color']),
                           showarrow=False))
    return annotations


def generate_barplot(text, annot_dict, orient='v', lmarg=120, h=400):
    """
    Generate the barplot with all data, using previous helper functions
    """
    layout = gen_layout(text[0], text[1], text[2], lmarg, h, gen_annotations(annot_dict))
    fig = go.Figure(data=gen_bars(barplot, 'color', orient=orient), layout=layout)
    return iplot(fig)